In [2]:
pip install openpyxl requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [102]:
import openpyxl
import requests
from bs4 import BeautifulSoup
import os



xlsx_file_path = 'Input.xlsx'
output_folder = './scraped_texts1/'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

wb = openpyxl.load_workbook(xlsx_file_path)
ws = wb.active

for row in ws.iter_rows(min_row=2, values_only=True):
    filename, url = row[0], row[1]
    output_file_path = os.path.join(output_folder, f"{filename}.txt")
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            title_tag = soup.find('title')
            title = title_tag.get_text(strip=True) if title_tag else 'Title Not Found'
        
        
            content = []
            content.append(title)
            dc=['td-post-content tagdiv-type','tdb-block-inner td-fix-index']
            p_strong = soup.find('div', class_='td-post-content tagdiv-type' )
            p_strong2= soup.find_all('div', class_='tdb-block-inner td-fix-index' )
            if(p_strong):
                text = p_strong.get_text(strip=True, separator='\n')
                content.append(text)
            elif(p_strong2):
                for p in p_strong2:
                
                    text2 = p.get_text(strip=True, separator='\n')

                    content.append(text2)
           
            text = '\n'.join(content)
            
            
            with open(output_file_path, 'w') as output_file:
                output_file.write(text)
            print(f'Successfully scraped and saved: {filename}.txt')
        else:
            print(f'Failed to retrieve {url}')
    except Exception as e:
        print(f'Error scraping {url}: {e}')

print('Scraping done.')


Successfully scraped and saved: blackassign0001.txt
Successfully scraped and saved: blackassign0002.txt
Successfully scraped and saved: blackassign0003.txt
Successfully scraped and saved: blackassign0004.txt
Successfully scraped and saved: blackassign0005.txt
Successfully scraped and saved: blackassign0006.txt
Successfully scraped and saved: blackassign0007.txt
Successfully scraped and saved: blackassign0008.txt
Successfully scraped and saved: blackassign0009.txt
Successfully scraped and saved: blackassign0010.txt
Successfully scraped and saved: blackassign0011.txt
Successfully scraped and saved: blackassign0012.txt
Successfully scraped and saved: blackassign0013.txt
Successfully scraped and saved: blackassign0014.txt
Successfully scraped and saved: blackassign0015.txt
Successfully scraped and saved: blackassign0016.txt
Successfully scraped and saved: blackassign0017.txt
Successfully scraped and saved: blackassign0018.txt
Successfully scraped and saved: blackassign0019.txt
Successfully

In [92]:
import os

# Path to the folder containing text files
folder_path = "./StopWords/"

# List to store paths of text files
text_files = []

# Iterate over all files in the folder
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if file_name.endswith('.txt') and os.path.isfile(file_path):
        text_files.append(file_path)

# Now text_files list contains paths to all text files in the folder
unique_stopwords = []

# Iterate over each stopwords file
for file_name in text_files:
    with open(file_name, 'r') as file:
        for line in file:
            word = line.strip()  
            if word:  
                unique_stopwords.append(word)

# Output file to store combined stopwords
output_file = 'combined_stopwords.txt'

# Write unique stopwords to the output file
with open(output_file, 'w') as file:
    for word in (unique_stopwords):
        file.write(word + '\n')

print(f"Combined stopwords written to {output_file}")


Combined stopwords written to combined_stopwords.txt


In [41]:
# Input file path
input_file_path = "combined_stopwords.txt"

# Output file path
output_file_path = "output.txt"

# Read input file and process each line
with open(input_file_path, 'r') as input_file:
    with open(output_file_path, 'w') as output_file:
        for line in input_file:
            
            parts = line.split('|')
            modified_line = parts[0]
            output_file.write(modified_line)

print("Words after '|' removed. Output written to", output_file_path)


Words after '|' removed. Output written to output.txt


In [9]:
import os
import string
# Function to remove stopwords from text
def remove_stopwords(text, stopwords):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_words)

# File paths
stopwords_file = "output.txt"
input_folder = "./scraped_texts1/"
output_folder = "./swscraped_texts1/"
stopwords_output_file = "stopwords_removed.txt"

# Read stopwords from file and store in a set
stopwords_set = set()
with open(stopwords_file, 'r') as f:
    for line in f:
        stopwords_set.update(line.split())

# Process each text file
for filename in os.listdir(input_folder):
    input_file_path = os.path.join(input_folder, filename)
    output_file_path = os.path.join(output_folder, filename)
    
    with open(input_file_path, 'r') as input_file:
        text = input_file.read()
        text=''.join(char for char in text if char not in string.punctuation)
        processed_text = remove_stopwords(text, stopwords_set)
        
        with open(output_file_path, 'w') as output_file:
            output_file.write(processed_text)

# Write stopwords to separate file
with open(stopwords_output_file, 'w') as f:
    f.write(' '.join(stopwords_set))

print("Stopwords removed from text files. Output written to", output_folder)
print("Stopwords written to", stopwords_output_file)


Stopwords removed from text files. Output written to ./swscraped_texts1/
Stopwords written to stopwords_removed.txt


In [11]:
import os
import string
import csv
import re
import nltk
from nltk.corpus import cmudict
from nltk.tokenize import word_tokenize


nltk.download('cmudict')


cmu_dict = cmudict.dict()
def count_personal_pronouns(text):
    pronoun_pattern = r'\b(?:I|we|my|ours|us)\b'
    regex = re.compile(pronoun_pattern, re.IGNORECASE)
    matches = regex.findall(text)
    filtered_matches = [match for match in matches if match.lower() != "us"]
    pronoun_count = len(filtered_matches)
    return pronoun_count

def count_syllables(word):
    
    if word.endswith(('es', 'ed')):
        return 0

    vowels = re.findall(r'[aeiouyAEIOUY]+', word)
    return len(vowels)
def count_syllables_in_text(text):
    words = text.split()
    syllable_counts = [count_syllables(word) for word in words]
    return syllable_counts

# Function to count syllables in a word
def count_syllables(word):
    phonemes = cmu_dict.get(word.lower())
    if phonemes:
        return len([ph for ph in phonemes[0] if ph[-1].isdigit()])
    else:
        return 0

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation

# Function to count complex words in a text
def count_complex_words(text):
    words = word_tokenize(text)
    complex_words = [word for word in words if count_syllables(word) > 2]
    return len(complex_words)

# Function to count positive and negative words in a text
def count_positive_negative(text, positive_words, negative_words):
    positive_count = sum(1 for word in text.split() if word.lower() in positive_words)
    negative_count = sum(1 for word in text.split() if word.lower() in negative_words)
    return positive_count, negative_count
def count_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return len(sentences)




# Read positive and negative words from text files
with open('positive-words.txt', 'r') as positive_file:
    positive_words = set(positive_file.read().split())

with open('negative-words.txt', 'r') as negative_file:
    negative_words = set(negative_file.read().split())

# Folder containing text files
folder_path = './swscraped_texts1/'

# Create and open the CSV file for writing
with open('Output Data Structure.csv', 'w', newline='') as csvfile:
    fieldnames = ['Filename', 'Positive score', 'Negative score','Polarity score','Subjectivity Score','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    # Iterate over each file in the folder
   
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
        
        # Read the content of the file
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                text_wp = remove_punctuation(text)
                words = text_wp.split()
    
                word_count = len(words)
                positive_count, negative_count = count_positive_negative(text, positive_words, negative_words)
                positive_count=int(positive_count)
                negative_count=int(negative_count)
                polarity_score=(positive_count-negative_count)/ ((positive_count + negative_count) + 0.000001)
                sub_score=(positive_count+negative_count)/ ((word_count) + 0.000001)
                complex_word_count = count_complex_words(text)
    
                sentence_count = count_sentences(text)
                avg_s_len=word_count/sentence_count
                pt_cw=complex_word_count /word_count
                fog_index=0.4*(avg_s_len+pt_cw)
                avg_num_wps=word_count/sentence_count
                syllable_counts = count_syllables_in_text(text)
                scpw=sum(syllable_counts)/word_count
                pronoun_count = count_personal_pronouns(text)
                total_length = sum(len(word) for word in words)
                avg_word_len=total_length/word_count
            

            # Write the counts to the CSV file
            writer.writerow({'Filename': filename, 'Positive score': positive_count, 'Negative score': negative_count,'Polarity score':polarity_score,'Subjectivity Score':sub_score,'AVG SENTENCE LENGTH':avg_s_len,'PERCENTAGE OF COMPLEX WORDS':pt_cw,'FOG INDEX':fog_index,'AVG NUMBER OF WORDS PER SENTENCE':avg_num_wps,'COMPLEX WORD COUNT':complex_word_count,'WORD COUNT':word_count,'SYLLABLE PER WORD':scpw,'PERSONAL PRONOUNS':pronoun_count,'AVG WORD LENGTH':avg_word_len,})


[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
